In [2]:
# Step 1: Import necessary libraries for file upload
from google.colab import files
import os

In [3]:
# Step 2: Manually upload the dataset file
uploaded = files.upload()

Saving labeled_telegram_data_conll.txt to labeled_telegram_data_conll.txt


In [4]:
# Use the file name from the uploaded dictionary
dataset_filename = list(uploaded.keys())[0]
print(f"Uploaded dataset: {dataset_filename}")

Uploaded dataset: labeled_telegram_data_conll.txt


In [5]:
# Step 3: Use the file path in your code
import pandas as pd

In [6]:
# Load the dataset into a Pandas DataFrame or appropriate structure
dataset_path = os.path.join(os.getcwd(), dataset_filename)
print(f"Dataset path: {dataset_path}")

Dataset path: /content/labeled_telegram_data_conll.txt


In [7]:
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch

In [8]:
# Function to load the dataset in CoNLL format
def load_conll_format(filepath):
    sentences = []
    labels = []
    with open(filepath, 'r', encoding='utf-8') as f:
        sentence = []
        label = []
        for line in f:
            if line == "\n":  # Sentence boundary
                if sentence:  # Check if sentence is not empty
                    sentences.append(sentence)
                    labels.append(label)
                sentence = []
                label = []
            else:
                token, tag = line.strip().split()
                sentence.append(token)
                label.append(tag)
    return sentences, labels

In [9]:
# Load the pre-trained tokenizer and model
model_checkpoint = "xlm-roberta-base"  # You can switch to "bert-tiny-amharic" if available
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
# Define label mapping
label_list = ['O', 'B-Product', 'I-Product', 'B-LOC', 'I-LOC', 'B-PRICE', 'I-PRICE']
label_to_id = {label: i for i, label in enumerate(label_list)}

In [11]:
# Function to tokenize the dataset and align labels with tokens
def tokenize_and_align_labels(batch):
    tokenized_inputs = tokenizer(batch['tokens'], truncation=True, is_split_into_words=True, padding=True)
    labels = []
    for i, label in enumerate(batch['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                aligned_labels.append(-100)  # Ignore subwords and special tokens
            elif word_idx != previous_word_idx:  # Beginning of a new word
                aligned_labels.append(label_to_id[label[word_idx]])
            else:
                aligned_labels.append(label_to_id[label[word_idx]])  # Continuation of the same word
            previous_word_idx = word_idx
        labels.append(aligned_labels)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
!pip install datasets

In [ ]:
!pip install transformers

In [14]:
# Import required libraries
from datasets import load_dataset
from transformers import AutoTokenizer

In [15]:
# Load the dataset (conll2003 for this example)
dataset = load_dataset("conll2003")

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [16]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [17]:
# Tokenize the dataset and align the labels
def tokenize_and_align_labels(batch):
    tokenized_inputs = tokenizer(batch['tokens'], truncation=True, is_split_into_words=True, padding=True)

    labels = []
    for i, label in enumerate(batch['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their corresponding word
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special token
            elif word_idx != previous_word_idx:  # First token of a word
                label_ids.append(label[word_idx])
            else:  # Label all tokens
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [18]:
# Apply the function to the dataset
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
# Print the tokenized datasets
print(tokenized_datasets)

In [20]:
# Set training arguments with matching save_strategy and evaluation_strategy
training_args = TrainingArguments(
    output_dir="./ner_model",
    evaluation_strategy="epoch",  # Evaluating at the end of each epoch
    save_strategy="epoch",  # Save the model at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True  # Load the best model based on evaluation metric at the end
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
# Load pre-trained model for token classification
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install evaluate

In [24]:
# Import required libraries
import torch
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForTokenClassification,
                          Trainer, TrainingArguments, DataCollatorForTokenClassification)
from evaluate import load as load_metric
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

In [23]:
# Load the dataset (conll2003 for this example)
dataset = load_dataset("conll2003", trust_remote_code=True)

In [ ]:
!pip install seqeval
!pip install

In [32]:
# Load the evaluation metric (seqeval for NER tasks)
from evaluate import load
from evaluate import load as load_metric
metric = load_metric("seqeval")

In [33]:
# Preprocessing the dataset
label_list = dataset['train'].features['ner_tags'].feature.names

In [34]:
# Tokenizer initialization
def tokenize_and_align_labels(examples, tokenizer, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their corresponding word
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special token
            elif word_idx != previous_word_idx:  # First token of a word
                label_ids.append(label[word_idx])
            else:  # Label all tokens
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [35]:
# Define models for comparison
model_names = [
    "xlm-roberta-base",  # XLM-Roberta
    "distilbert-base-multilingual-cased",  # DistilBERT
    "bert-base-multilingual-cased"  # mBERT
]

In [36]:
# Load evaluation metric (for example: accuracy)
from evaluate import load
metric = load("accuracy")

In [37]:
# Preprocess dataset function
def tokenize_and_align_labels(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
!pip install evaluate

In [58]:
import numpy as np
from evaluate import load  # Corrected import for load_metric
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report

# Fine-tuning function for models
def model_fine_tune(model_name, dataset, label_list, num_epochs=3):
    print(f"\nFine-tuning {model_name}...")

    # Initialize tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))

    # Tokenize dataset
    tokenized_datasets = dataset.map(lambda x: tokenize_and_align_labels(x, tokenizer), batched=True)

    # Data collator
    data_collator = DataCollatorForTokenClassification(tokenizer)

    # Load evaluation metric
    metric = load_metric("seqeval")

    # Define compute_metrics function for evaluation
    def compute_metrics(p):
        predictions, labels = p
        predictions = np.argmax(predictions, axis=2)

        # Remove ignored index (-100)
        true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
        true_predictions = [[label_list[p] for (p, l) in zip(pred, label) if l != -100] for pred, label in zip(predictions, labels)]

        results = metric.compute(predictions=true_predictions, references=true_labels)
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"]
        }

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=f"./results-{model_name}",
        evaluation_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=50
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['validation'],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_result = trainer.evaluate()
    print(f"Evaluation results for {model_name}: {eval_result}")

    # Perform predictions on the test set for classification report
    predictions, labels, _ = trainer.predict(tokenized_datasets['test'])
    preds = np.argmax(predictions, axis=2)

    # Remove ignored index (-100) and flatten the predictions/labels
    true_preds = [[p for (p, l) in zip(pred, label) if l != -100] for pred, label in zip(preds, labels)]
    true_labels = [[l for (p, l) in zip(pred, label) if l != -100] for pred, label in zip(preds, labels)]

    # Flatten for classification report
    flat_true_preds = [item for sublist in true_preds for item in sublist]
    flat_true_labels = [item for sublist in true_labels for item in sublist]

    # Generate classification report
    report = classification_report(flat_true_labels, flat_true_preds, target_names=label_list)
    print(f"\nClassification Report for {model_name}:\n", report)


# Load labels for NER task (from dataset)
label_list = dataset['train'].features['ner_tags'].feature.names

# Fine-tune and compare models
for model_name in model_names:
    model_fine_tune(model_name, dataset, label_list, num_epochs=3)



Fine-tuning xlm-roberta-base...


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.053900,0.045651,0.914305,0.928307,0.921253,0.987462
2,0.034400,0.040047,0.940323,0.949344,0.944812,0.990285
3,0.016900,0.037322,0.945598,0.950690,0.948137,0.991024


Evaluation results for xlm-roberta-base: {'eval_loss': 0.03732157498598099, 'eval_precision': 0.9455975895547372, 'eval_recall': 0.9506900033658701, 'eval_f1': 0.9481369587109767, 'eval_accuracy': 0.9910244928157004, 'eval_runtime': 12.148, 'eval_samples_per_second': 267.534, 'eval_steps_per_second': 8.396, 'epoch': 3.0}

Classification Report for xlm-roberta-base:
               precision    recall  f1-score   support

           O       1.00      0.99      1.00     38323
       B-PER       0.96      0.96      0.96      1617
       I-PER       0.98      0.99      0.98      1156
       B-ORG       0.89      0.92      0.90      1661
       I-ORG       0.87      0.93      0.90       835
       B-LOC       0.94      0.93      0.93      1668
       I-LOC       0.85      0.87      0.86       257
      B-MISC       0.82      0.85      0.84       702
      I-MISC       0.69      0.78      0.73       216

    accuracy                           0.98     46435
   macro avg       0.89      0.91  

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

...

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.048900,0.039363,0.933989,0.942949,0.938447,0.989525
2,0.029000,0.039337,0.948163,0.951195,0.949677,0.991083
3,0.012300,0.033003,0.956296,0.961124,0.958704,0.992407


Evaluation results for bert-base-multilingual-cased: {'eval_loss': 0.0330033004283905, 'eval_precision': 0.9562960482250502, 'eval_recall': 0.9611242006058566, 'eval_f1': 0.9587040456605673, 'eval_accuracy': 0.9924068377399634, 'eval_runtime': 11.2332, 'eval_samples_per_second': 289.321, 'eval_steps_per_second': 9.08, 'epoch': 3.0}

Classification Report for bert-base-multilingual-cased:
               precision    recall  f1-score   support

           O       1.00      0.99      0.99     38323
       B-PER       0.97      0.95      0.96      1617
       I-PER       0.99      0.99      0.99      1156
       B-ORG       0.89      0.91      0.90      1661
       I-ORG       0.88      0.91      0.90       835
       B-LOC       0.93      0.93      0.93      1668
       I-LOC       0.81      0.94      0.87       257
      B-MISC       0.82      0.85      0.83       702
      I-MISC       0.64      0.76      0.70       216

    accuracy                           0.98     46435
   macro avg